In [22]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import time
import math
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

from sklearn.metrics import r2_score
%matplotlib inline

In [23]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        result[feature_name] = (df[feature_name] - df[feature_name].mean()) / df[feature_name].std()
    return result

In [24]:
#Importing data
data = pd.read_csv('../data/df_final.csv')
data = data.drop(columns=['lat', 'lon', 'elv', 'classid', 'c4', 'koeppen_code', 'igbp_land_use'])
# Drop AR-Vir and CN-Cng
data = data[data.sitename != "AR-Vir"]
data = data[data.sitename != "CN-Cng"]

In [25]:
data = pd.get_dummies(data, columns=['plant_functional_type'])

In [26]:
sites = data['sitename'].unique()
sites_df = [data[data['sitename'] == site] for site in sites]

for i in range(len(sites_df)):
    sites_df[i]['date'] = pd.to_datetime(sites_df[i]['date'], format="%Y-%m-%d")
    sites_df[i] = sites_df[i].set_index("date")
    sites_df[i] = sites_df[i].drop(columns=["sitename"])
    sites_df[i] = sites_df[i].reset_index(drop=True)

<ipython-input-26-3bb9fd8c6aed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sites_df[i]['date'] = pd.to_datetime(sites_df[i]['date'], format="%Y-%m-%d")


In [27]:
x_time_dep_cols = ['TA_F', 'TA_F_DAY', 'TA_F_NIGHT', 'SW_IN_F', 'LW_IN_F', 'VPD_F', 'PA_F', 'P_F', 'WS_F', 'LE_F_MDS', 'NEE_VUT_REF', 'wscal', 'fpar', 'whc']
x_time_invariant_cols = ['plant_functional_type_Cereal crop',
       'plant_functional_type_Deciduous Broadleaf Trees',
       'plant_functional_type_Evergreen Broadleaf Trees',
       'plant_functional_type_Evergreen Needleleaf Trees',
       'plant_functional_type_Grass', 'plant_functional_type_Shrub',
       'plant_functional_type_Water']
x_cols = x_time_dep_cols + x_time_invariant_cols
y_col = ['GPP_NT_VUT_REF']

In [29]:
#define data
train_sites = list(range(40))
test_sites = [62]

train_dfs = [sites_df[i] for i in train_sites]
test_dfs = [sites_df[i] for i in test_sites]


X_train = [pd.concat([normalize(df[x_time_dep_cols]), df[x_time_invariant_cols]], axis=1).values for df in train_dfs]
conditional_train = [torch.tensor(df[x_time_invariant_cols].values, dtype=torch.float32) for df in train_dfs]
y_train = [normalize(df[y_col]).values for df in train_dfs]

X_test = [pd.concat([normalize(df[x_time_dep_cols]), df[x_time_invariant_cols]], axis=1).values for df in test_dfs]
conditional_test = [torch.tensor(df[x_time_invariant_cols].values, dtype=torch.float32) for df in test_dfs]
y_test = [normalize(df[y_col]).values for df in test_dfs]

In [30]:
# VAE model
class Encoder(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.input_features = input_features
        self.output_features = output_features

        self.rnn = nn.LSTM(input_size=input_features, hidden_size=output_features)
    
    def forward(self, x):
        outputs, (h, c) = self.rnn(x)

        return outputs.squeeze(1) #shape=(seq_len, num_dir * output_features)

class Reparametrize(nn.Module):
    def __init__(self, encoder_output, latent_size):
        super().__init__()
        self.encoder_output = encoder_output
        self.latent_size = latent_size

        self.fc_to_mean = nn.Linear(encoder_output, latent_size)
        self.fc_to_logvar = nn.Linear(encoder_output, latent_size)

        nn.init.xavier_uniform_(self.fc_to_mean.weight)
        nn.init.xavier_uniform_(self.fc_to_logvar.weight)

    def forward(self, x):
        self.mean = self.fc_to_mean(x)
        self.logvar = self.fc_to_logvar(x)

        std = torch.exp(0.5 * self.logvar)
        eps = torch.randn_like(std)
        z = eps.mul(std).add_(self.mean)
        return z, self.mean, self.logvar

class Decoder(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.input_features = input_features
        self.output_features = output_features

        self.rnn = nn.LSTM(input_size=input_features, hidden_size=output_features)

        self.rnn_to_output = nn.Sequential(
            nn.Linear(self.output_features, self.output_features),
            nn.Tanh(),
            nn.Linear(self.output_features, self.output_features),
        )

    def forward(self, x):
        outputs, (h, c) = self.rnn(x)
        outputs = outputs.squeeze(1)
        return self.rnn_to_output(outputs) #shape=(seq_len, num_dir * output_features)

class Regressor(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.input_features = input_features
        self.output_features = output_features

        self.rnn = nn.LSTM(input_size=input_features, hidden_size=output_features)

        self.rnn_to_output = nn.Sequential(
            nn.Linear(self.output_features, self.output_features),
            nn.Tanh(),
            nn.Linear(self.output_features, 1),
        )

    def forward(self, x):
        outputs, (h, c) = self.rnn(x)
        outputs = outputs.squeeze(1)
        return self.rnn_to_output(outputs) #shape=(seq_len, num_dir * output_features)

class Model(nn.Module):
    def __init__(self, encoder, decoder, reparam):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.reparam = reparam

    def forward(self, x, conditional):
        x = self.encoder(x) # x has the conditional
        z, mean, logvar = self.reparam(x)

        # concat the conditoinal to z
        z1 = torch.cat([z, conditional], dim=1)
        
        # decode
        x = self.decoder(z1.unsqueeze(1))

        return x, mean, logvar

In [31]:
#loss functions
def loss_fn(x_decoded, x, mu, logvar, w):
    kl_loss = w * (-0.5) * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    recon_loss = F.mse_loss(x_decoded, x)
    return kl_loss + recon_loss, kl_loss, recon_loss

In [ ]:
!nvidia-smi

In [17]:
#define parameter and model (this model should be ran on gpu)
ENCODER_INPUT_DIM = len(x_cols)
ENCODER_OUTPUT_DIM = 16
REPARAM_INPUT_DIM = ENCODER_OUTPUT_DIM
LATENT_DIM = 2
REPARAM_OUTPUT_DIM = LATENT_DIM
DECODER_INPUT_DIM = LATENT_DIM + len(x_time_invariant_cols)
DECODER_OUTPUT_DIM = len(x_time_dep_cols)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

encoder = Encoder(ENCODER_INPUT_DIM, ENCODER_OUTPUT_DIM).to(DEVICE)
decoder = Decoder(DECODER_INPUT_DIM, DECODER_OUTPUT_DIM).to(DEVICE)
reparam = Reparametrize(REPARAM_INPUT_DIM, REPARAM_OUTPUT_DIM).to(DEVICE)
model = Model(encoder, decoder, reparam).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

RuntimeError: CUDA error: out of memory

In [ ]:
# Training

EPOCHS = 200
for epoch in range(EPOCHS):
    train_loss = 0.0
    train_kl_loss = 0.0
    train_recon_loss = 0.0
    test_loss = 0.0
    test_kl_loss = 0.0
    test_recon_loss = 0.0


    model.train()
    for (x, c, y) in zip(X_train, conditional_train, y_train):
        # Convert to tensors
        x = torch.FloatTensor(x).unsqueeze(1).to(DEVICE)
        c = torch.FloatTensor(c).to(DEVICE)
        y = torch.FloatTensor(y).to(DEVICE)

        # Get predictions
        out, mean, logvar = model(x, c)

        # Remove the conditional from x
        x = x.squeeze(1)[:, :len(x_time_dep_cols)]

        # Get loss and update
        optimizer.zero_grad()
        loss, kl_loss, recon_loss = loss_fn(out, x, mean, logvar, 1)
        loss.backward()
        optimizer.step()
    
        # Update losses
        train_loss += loss.item()
        train_kl_loss += kl_loss.item()
        train_recon_loss += recon_loss.item()

    model.eval()
    with torch.no_grad():
        for (x, c, y) in zip(X_test, conditional_test, y_test):
            # Convert to tensors
            x = torch.FloatTensor(x).unsqueeze(1).to(DEVICE)
            c = torch.FloatTensor(c).to(DEVICE)
            y = torch.FloatTensor(y).to(DEVICE)

            # Get predictions
            out, mean, logvar = model(x, c)

            # Remove the conditional from x
            x = x.squeeze(1)[:, :len(x_time_dep_cols)]

            # Get loss and update
            loss, kl_loss, recon_loss = loss_fn(out, x, mean, logvar, 1)
        
            # Update losses
            test_loss += loss.item()
            test_kl_loss += kl_loss.item()
            test_recon_loss += recon_loss.item()

    

    train_loss /= len(X_train)
    train_kl_loss /= len(X_train)
    train_recon_loss /= len(X_train)
    test_loss /= len(X_test)
    test_kl_loss /= len(X_test)
    test_recon_loss /= len(X_test)
    
    print(f"Epoch: {epoch+1}/{EPOCHS}")
    print(f"Train loss: {train_loss:.4f} | Recon loss: {train_recon_loss:.4f} | KL loss: {train_kl_loss:.4f}")
    print(f"Test loss: {test_loss:.4f} | Recon loss: {test_recon_loss:.4f} | KL loss: {test_kl_loss:.4f}")

In [ ]:
# Visualization of the result 
from sklearn.decomposition import PCA

def visualize_recon(model, x, c, col):
    out, mean, logvar = model(x, c)
    out = out.detach().cpu().numpy()
    mean = mean.detach().cpu().numpy()
    logvar = logvar.detach().cpu().numpy()
    x = x.detach().cpu().numpy()
    c = c.detach().cpu().numpy()

    x = x.squeeze(1)[:, :len(x_time_dep_cols)]

    x_axis = range(x.shape[0])
    plt.figure(dpi=200)
    plt.scatter(x=x_axis, y=x[:, col], label='GT', s=1)
    plt.scatter(x=x_axis, y=out[:, col], label='Preds', s=1)
    plt.legend()
    plt.show()

def visualize_latent_1(model, x, c, months, columns):
    out, mean, logvar = model(x, c)
    mean = mean.detach().cpu().numpy()

    # pca = PCA(n_components=2)
    # mean = pca.fit_transform(mean)
    plt.figure(dpi=200)

    summer_months = np.argwhere((months == 6) | (months == 7) | (months == 8)).flatten()
    winter_months = np.argwhere((months == 1) | (months == 2) | (months == 12)).flatten()
    spring_months = np.argwhere((months == 3) | (months == 4) | (months == 5)).flatten()
    fall_months = np.argwhere((months == 9) | (months == 10) | (months == 11)).flatten()
    
    summer_mean = mean[summer_months, :]
    winter_mean = mean[winter_months, :]
    spring_mean = mean[spring_months, :]
    fall_mean = mean[fall_months, :]
    plt.scatter(x=summer_mean[:, 0], y=summer_mean[:, 1], s=1, c='red')
    plt.scatter(x=winter_mean[:, 0], y=winter_mean[:, 1], s=1, c='blue')
    plt.scatter(x=spring_mean[:, 0], y=spring_mean[:, 1], s=1, c='yellow')
    plt.scatter(x=fall_mean[:, 0], y=fall_mean[:, 1], s=1, c='orange')
    
    plt.title("Latent space")
    plt.show()

def visualize_latent_2(model, x, c, y, columns):
    out, mean, logvar = model(x, c)
    mean = mean.detach().cpu().numpy()

    # pca = PCA(n_components=2)
    # mean = pca.fit_transform(mean)
    plt.figure(dpi=200)
    plt.scatter(x=mean[:, 0], y=mean[:, 1], s=1, c=y)
    
    plt.title("Latent space")
    plt.show()

def visualize_gpp(model, x, c, y):
    out, mean, logvar = model(x, c)
    out = out.detach().cpu().numpy()
    x_axis = range(out.shape[0])
    y = y.detach().cpu().numpy()

    plt.figure(dpi=200)
    plt.scatter(x=x_axis, y=out, s=1, label="Preds")
    plt.scatter(x=x_axis, y=y, s=1, label="GT")
    plt.legend()
    plt.show()
    

In [ ]:
sites_with_dates_df = [data[data['sitename'] == site] for site in sites]

def convert_to_months(values):
    mapping = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun', '07': 'Jul', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12':'Dec'}
    mapping = {'01': 1, '02': 2, '03': 3, '04': 4, '05': 5, '06': 6, '07': 7, '08': 8, '09': 9, '10': 10, '11': 11, '12': 12}
    months = [mapping[v.split('-')[1]] for v in values]
    return np.array(months)

In [ ]:
np.argwhere(sites=='IT-Tor')

In [18]:
# output plot for paper(Figure 7)
site = 41
col = 0
months = convert_to_months(sites_with_dates_df[site].date.values)

x = torch.FloatTensor(X_train[site]).unsqueeze(1).to(DEVICE)
c = torch.FloatTensor(conditional_train[site]).to(DEVICE)
y = torch.FloatTensor(y_train[site]).to(DEVICE)
# visualize_recon(model, x, c, col)
# visualize_latent_1(model, x, c, months, x_cols)
visualize_latent_1(model, x, c, months, x_cols)
visualize_latent_2(model, x, c, y_train[site], x_cols)
# visualize_gpp(model, x, c, y)

NameError: name 'convert_to_months' is not defined